## Análise de sentimentos em dataset com reviews de filmes

### Problema: 

* dado um conjunto de dados classificado em `positivo` e `negativo`, treinar um modelo que faz análise de sentimento.

### Solução proposta:

Os dados contêm *reviews* em diversas línguas, mas quase 98% delas são em inglês, segundo detecção de línguas feita através do [`langdetect`](https://github.com/Mimino666/langdetect) (ver tabela abaixo), então o primeiro passo será remover as demais línguas (pois elas gerariam ruído no treinamento) e focar na análise de sentimentos das sentenças que estão em inglês.

![Tabela](data/table_langs.png)

A solução que proponho pode ser dividida em quatro passos:

* tratamento dos dados:
    * normalização do texto (remoção de eventuais acentos/pontuação e stopwords)
    * inspirada em ideias contidas [nesse artigo](https://oasys.umiacs.umd.edu/oasysnew/papers/icwsmV2.pdf) e [nesse outro artigo](http://people.cs.pitt.edu/~wiebe/courses/CS3730/Fall08/chesleyetal2005.pdf), a ideia é fazer part-of-speech tagging com o módulo [`spacy`](https://spacy.io/), que é [mais rápido para fazer o pos-tag](http://blog.thedataincubator.com/2016/04/nltk-vs-spacy-natural-language-processing-in-python/) do que o módulo tradicional de nlp [`nltk`](http://www.nltk.org/), para filtrar das sentenças somente:
        * os adjetivos;
        * os adjetivos e advérbios;
        * os adjetivos e verbos;
        * etc., para comparar com o approach que usa o texto todo.

* divisão do conjunto de dados em `k` folds (a escolher, mas poderia ser o padrão `k=10`, com `shuffle`, já que o tamanho do conjunto de dados é relativamente pequeno), para fazer validação cruzada, com 75% dos dados para treinamento e 25% para teste

* treinamento do modelo de classificação usando a ferramenta [`fasttext`](https://github.com/facebookresearch/fastText), através da [interface](https://github.com/salestock/fastText.py) dela para `Python`:
    * `fasttext` é uma ferramenta desenvolvida para criar representações vetoriais e classificar textos, que tem performance similar a algoritmos de deep learning, mas com tempo de treinamento significativamente menor (de acordo com testes explorados no artigo [*Bag of Tricks for Efficient Text Classification*](https://arxiv.org/pdf/1607.01759v2.pdf)) e [uso de memória reduzido](https://code.facebook.com/posts/162370720961195/expanded-fasttext-library-now-fits-on-smaller-memory-devices/)
    * como faz uso das subpalavras, explora melhor a estrutura das palavras de uma língua, obtendo [melhor performance em testes envolvendo analogia sintática](https://rare-technologies.com/fasttext-and-gensim-word-embeddings/) entre palavras.
    * aceita [vetores pré-treinados usando bases maiores](https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md), o que pode ser uma vantagem, dado o conjunto pequeno de dados que estamos lidando neste exercício

* validação dos dados:
    * cálculo de scores como acurácia, precisão, recall e f-score
    * cálculo da matriz de confusão